In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple
import keras

data = pd.read_csv('train.csv')

/home/blair/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/blair/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
processed_data = data
processed_data['Pclass'] = data['Pclass']/10
processed_data['Age'] = data['Age']/100

for m in processed_data['Sex']:
    if m == 'male':
        processed_data[m] = 1
        print(processed_data[m])
    else:
        m=0
processed_data

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    1
870    1
871    1
872    1
873    1
874    1
875    1
876    1
877    1
878    1
879    1
880    1
881    1
882    1
883    1
884    1
885    1
886    1
887    1
888    1
889    1
890    1
Name: male, Length: 891, dtype: int64
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    1
870    1
871    1
872    1
873    1
874    1
875    1

Name: male, Length: 891, dtype: int64
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    1
870    1
871    1
872    1
873    1
874    1
875    1
876    1
877    1
878    1
879    1
880    1
881    1
882    1
883    1
884    1
885    1
886    1
887    1
888    1
889    1
890    1
Name: male, Length: 891, dtype: int64
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    1
870    1
871   

Name: male, Length: 891, dtype: int64
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    1
870    1
871    1
872    1
873    1
874    1
875    1
876    1
877    1
878    1
879    1
880    1
881    1
882    1
883    1
884    1
885    1
886    1
887    1
888    1
889    1
890    1
Name: male, Length: 891, dtype: int64
0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
861    1
862    1
863    1
864    1
865    1
866    1
867    1
868    1
869    1
870    1
871   

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,male
0,1,0,0.3,"Braund, Mr. Owen Harris",male,0.22,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,0.1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,0.38,1,0,PC 17599,71.2833,C85,C,1
2,3,1,0.3,"Heikkinen, Miss. Laina",female,0.26,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,0.1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,0.35,1,0,113803,53.1000,C123,S,1
4,5,0,0.3,"Allen, Mr. William Henry",male,0.35,0,0,373450,8.0500,NaN,S,1
5,6,0,0.3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,1
6,7,0,0.1,"McCarthy, Mr. Timothy J",male,0.54,0,0,17463,51.8625,E46,S,1
7,8,0,0.3,"Palsson, Master. Gosta Leonard",male,0.02,3,1,349909,21.0750,NaN,S,1
8,9,1,0.3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,0.27,0,2,347742,11.1333,NaN,S,1
9,10,1,0.2,"Nasser, Mrs. Nicholas (Adele Achem)",female,0.14,1,0,237736,30.0708,NaN,C,1


In [5]:
sample = np.random.choice(processed_data.index, size=int(len(processed_data)*0.9), replace=False)
train_data, test_data = processed_data.iloc[sample], processed_data.drop(sample)

print("Number of training samples is", len(train_data))
print("Number of testing samples is", len(test_data))
print(train_data[:10])
print(test_data[:10])


Number of training samples is 801
Number of testing samples is 90
     PassengerId  Survived  Pclass  \
530          531         1     0.2   
82            83         1     0.3   
621          622         1     0.1   
109          110         1     0.3   
289          290         1     0.3   
670          671         1     0.2   
19            20         1     0.3   
142          143         1     0.3   
792          793         0     0.3   
527          528         0     0.1   

                                                  Name     Sex   Age  SibSp  \
530                           Quick, Miss. Phyllis May  female  0.02      1   
82                      McDermott, Miss. Brigdet Delia  female   NaN      0   
621                       Kimball, Mr. Edwin Nelson Jr    male  0.42      1   
109                                Moran, Miss. Bertha  female   NaN      1   
289                               Connolly, Miss. Kate  female  0.22      0   
670  Brown, Mrs. Thomas William Solomon (

In [6]:
features = np.array(train_data.drop('Survived', axis=1))
targets = np.array(keras.utils.to_categorical(train_data['Survived'], 2))
features_test = np.array(test_data.drop('Survived', axis=1))
targets_test = np.array(keras.utils.to_categorical(test_data['Survived'], 2))

print(features[:10])
print(targets[:10])

[[531 0.2 'Quick, Miss. Phyllis May' 'female' 0.02 1 1 '26360' 26.0 nan
  'S' 1]
 [83 0.3 'McDermott, Miss. Brigdet Delia' 'female' nan 0 0 '330932'
  7.7875 nan 'Q' 1]
 [622 0.1 'Kimball, Mr. Edwin Nelson Jr' 'male' 0.42 1 0 '11753' 52.5542
  'D19' 'S' 1]
 [110 0.3 'Moran, Miss. Bertha' 'female' nan 1 0 '371110' 24.15 nan 'Q' 1]
 [290 0.3 'Connolly, Miss. Kate' 'female' 0.22 0 0 '370373' 7.75 nan 'Q'
  1]
 [671 0.2 'Brown, Mrs. Thomas William Solomon (Elizabeth Catherine Ford)'
  'female' 0.4 1 1 '29750' 39.0 nan 'S' 1]
 [20 0.3 'Masselmani, Mrs. Fatima' 'female' nan 0 0 '2649' 7.225 nan 'C'
  1]
 [143 0.3 'Hakkarainen, Mrs. Pekka Pietari (Elin Matilda Dolck)' 'female'
  0.24 1 0 'STON/O2. 3101279' 15.85 nan 'S' 1]
 [793 0.3 'Sage, Miss. Stella Anna' 'female' nan 8 2 'CA. 2343' 69.55 nan
  'S' 1]
 [528 0.1 'Farthing, Mr. John' 'male' nan 0 0 'PC 17483' 221.7792 'C95'
  'S' 1]]
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [7]:
# Imports

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

# Building the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(6,)))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(2, activation='softmax'))

# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
Total params: 9,282
Trainable params: 9,282
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(features, targets, epochs=200, batch_size=100, verbose=0)

ValueError: Error when checking input: expected dense_1_input to have shape (6,) but got array with shape (12,)